In [43]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
from sklearn.linear_model import LinearRegression
import ast

def csv_reader(filename, datatyper):
    
    
    # Les CSV med korrekt format
    df = pd.read_csv(filename, header=None, names=['datatype', 'value','unit', 'timeOffset', 'timeResolution', 'timeSeriesId', 'performanceCategory', 'qualityCode', '..', 'station', 'referenceTimestamp'])
    

    
    # Sjekk at nødvendige kolonner finnes
    if not {'datatype', 'value','unit', 'timeOffset', 'timeResolution', 'timeSeriesId', 'performanceCategory', 'qualityCode', '..', 'station', 'referenceTimestamp'}.issubset(df.columns):
        raise ValueError("CSV-filen har ikke forventede kolonner: 'datatype', 'value','unit', 'timeOffset', 'timeResolution', 'timeSeriesId', 'performanceCategory', 'qualityCode', '..', 'station', 'referenceTimestamp'")
    
    records = []
    
    for _, row in df.iterrows():
        datatype = row['datatype']
        value = row['value']
        unit = row['unit']
        station = row['station']
        timestamp = (row['referenceTimestamp'])[:10]
        
        
        records.append({
                    'referenceTimestamp': timestamp,
                    'datatype': datatype,
                    'value': value,
                    'unit': unit,
                    'station' : station
                })
    result_df = pd.DataFrame(records)
    if result_df.empty:
        return {}
    
    
    # Konverter timestamp til datetime og sørger for at alle verdier under 'value' er tall

    result_df['value'] = pd.to_numeric(result_df['value'], errors='coerce')
    result_df['referenceTimestamp'] = pd.to_datetime(result_df['referenceTimestamp'], errors='coerce')
    result_df = result_df.dropna(subset=['referenceTimestamp'])  # Fjern rader med ugyldig tid
        
        
    # Fjern store avvik ved hjelp av Z-score
    result_df['value'] = result_df.groupby('datatype')['value'].transform(
        lambda x: x.where(np.abs(zscore(x)) < 3)
    )
    
    
    # Fyll inn manglende verdier ved hjelp av lineær regresjon
    for datatype in datatyper:
        subset = result_df[result_df['datatype'] == datatype].copy()
        subset = subset.sort_values('referenceTimestamp')
        
        # Konverter tid til numerisk verdi for regresjon
        subset['time_numeric'] = (subset['referenceTimestamp'] - subset['referenceTimestamp'].min()).dt.total_seconds()
        
        missing_mask = subset['value'].isna()
        if missing_mask.any() and not subset['value'].isna().all():
            reg = LinearRegression()
            known_x = subset.loc[~missing_mask, 'time_numeric'].values.reshape(-1, 1)
            known_y = subset.loc[~missing_mask, 'value'].values
            reg.fit(known_x, known_y)
            
            pred_x = subset.loc[missing_mask, 'time_numeric'].values.reshape(-1, 1)
            subset.loc[missing_mask, 'value'] = reg.predict(pred_x)
        
        result_df.update(subset)  
        
        
    
    #Uferdig kode. Skal legge sammen nedbørsmengde for hver dag og får formatert om til dictionary med datatype som nøkkel

    print(result_df)
    #print(result_df)  
    # Konverter til ønsket format
    
    result = {}
    for _,row in result_df.iterrows():]
        while row['referenceTimestamp']
        
        filtered_df = result_df[result_df['datatype'] == datatype]
        result[datatype] = {
            row['referenceTimestamp']: {'value': row['value'], 'unit': row['unit']}
            for _, row in filtered_df.iterrows()
        }
        
    return result




print(csv_reader('/Users/kristiansolberg1/Library/CloudStorage/OneDrive-NTNU/Anvendt programmering/Mappeprosjekt/miljodataAnalyse-1/data/met_data.csv', 'sum(precipitation_amount P1D)'))


      referenceTimestamp                       datatype  value   unit  \
0             2015-01-01  sum(precipitation_amount P1D)    0.2     mm   
1             2015-01-01  sum(precipitation_amount P1D)    0.0     mm   
2             2015-01-01  sum(duration_of_sunshine P1D)    0.0  hours   
3             2015-01-02  sum(precipitation_amount P1D)    0.5     mm   
4             2015-01-02  sum(precipitation_amount P1D)    0.7     mm   
...                  ...                            ...    ...    ...   
21904         2024-12-30  sum(precipitation_amount P1D)    7.1     mm   
21905         2024-12-30  sum(duration_of_sunshine P1D)    0.0  hours   
21906         2024-12-31  sum(precipitation_amount P1D)    4.2     mm   
21907         2024-12-31  sum(precipitation_amount P1D)   14.9     mm   
21908         2024-12-31  sum(duration_of_sunshine P1D)    0.0  hours   

         station  
0      SN18700:0  
1      SN18700:0  
2      SN18700:0  
3      SN18700:0  
4      SN18700:0  
...      